In [1]:
%tensorflow_version 1.x
%pip install colorama

TensorFlow 1.x selected.


In [2]:
import numpy as np
import tensorflow as tf
import random
import colorama
from colorama import Fore, Style


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/'Colab Notebooks'/'SentiGAN'/'Real_dataset'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/SentiGAN/Real_dataset


In [4]:
#%cd 'Real_dataset'

In [5]:
from dataloader import Gen_Data_loader, Dis_Data_loader
import pickle
from generator import Generator
from discriminator import Discriminator

In [6]:
print(tf.__version__)

1.15.2


In [7]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [8]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15025864815814256789, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1693946607371587146
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 12025559358341955220
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14912199066
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17254278464094397573
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [9]:

#########################################################################################
#  Generator  Hyper-parameters
######################################################################################
EMB_DIM = 200 # embedding dimension
HIDDEN_DIM = 200 # hidden state dimension of lstm cell
MAX_SEQ_LENGTH = 17  # max sequence length
BATCH_SIZE = 64


#########################################################################################
#  Discriminator  Hyper-parameters
#########################################################################################
dis_embedding_dim = 64
dis_filter_sizes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15]
dis_num_filters = [100, 200, 200, 200, 200, 100, 100, 100, 100, 100, 160]
dis_dropout_keep_prob = 0.75
dis_l2_reg_lambda = 0.2
dis_batch_size = 64


In [10]:
#########################################################################################
#  Basic Training Parameters
#########################################################################################
TOTAL_BATCH = 2000
dataset_path = "../../data/movie/"
emb_dict_file = dataset_path + "imdb_word.vocab"

# imdb corpus
imdb_file_txt = dataset_path + "imdb/imdb_sentences.txt"
imdb_file_id = dataset_path + "imdb/imdb_sentences.id"

# sstb corpus
sst_pos_file_txt = dataset_path + 'sstb/sst_pos_sentences.txt'
sst_pos_file_id = dataset_path + 'sstb/sst_pos_sentences.id'
sst_neg_file_txt = dataset_path + 'sstb/sst_neg_sentences.txt'
sst_neg_file_id = dataset_path + 'sstb/sst_neg_sentences.id'


eval_file = 'save/eval_file.txt'
eval_text_file = 'save/eval_text_file.txt'
negative_file = 'save/generator_sample.txt'
infer_file = 'save/infer/'


In [11]:
def generate_samples(sess, trainable_model, generated_num, output_file, vocab_list, if_log=False, epoch=0):
    # Generate Samples
    generated_samples = []
    for _ in range(int(generated_num)):
        generated_samples.extend(trainable_model.generate(sess))

    if if_log:
        mode = 'a'
        if epoch == 0:
            mode = 'w'
        with open(eval_text_file, mode) as fout:
            # id_str = 'epoch:%d ' % epoch
            for poem in generated_samples:
                poem = list(poem)
                if 2 in poem:
                    poem = poem[:poem.index(2)]
                buffer = ' '.join([vocab_list[x] for x in poem]) + '\n'
                fout.write(buffer)

    with open(output_file, 'w') as fout:
        for poem in generated_samples:
            poem = list(poem)
            if 2 in poem:
                poem = poem[:poem.index(2)]
            buffer = ' '.join([str(x) for x in poem]) + '\n'
            fout.write(buffer)


def generate_infer(sess, trainable_model, epoch, vocab_list):
    generated_samples = []
    for _ in range(int(100)):
        # generated_samples.extend(trainable_model.infer(sess))
        generated_samples.extend(trainable_model.generate(sess))
    file = infer_file+str(epoch)+'.txt'
    with open(file, 'w') as fout:
        for poem in generated_samples:
            poem = list(poem)
            if 2 in poem:
                poem = poem[:poem.index(2)]
            buffer = ' '.join([vocab_list[x] for x in poem]) + '\n'
            fout.write(buffer)
    print("%s saves" % file)
    return


def produce_samples(generated_samples):
    produces_sample = []
    for poem in generated_samples:
        poem_list = []
        for ii in poem:
            if ii == 0:  # _PAD
                continue
            if ii == 2:  # _EOS
                break
            poem_list.append(ii)
        produces_sample.append(poem_list)
    return produces_sample


def load_emb_data(emb_dict_file):
    word_dict = {}
    word_list = []
    item = 0
    with open(emb_dict_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            word = line.strip()
            word_dict[word] = item
            item += 1
            word_list.append(word)
    length = len(word_dict)
    print("Load embedding success! Num: %d" % length)
    return word_dict, length, word_list


def pre_train_epoch(sess, trainable_model, data_loader):
    # Pre-train the generator using MLE for one epoch
    supervised_g_losses = []
    data_loader.reset_pointer()

    for it in range(200):  # data_loader.num_batch):
        batch = data_loader.next_batch()
        _, g_loss = trainable_model.pretrain_step(sess, batch)
        supervised_g_losses.append(g_loss)

    return np.mean(supervised_g_losses)


In [12]:
vocab_dict, vocab_size, vocab_list = load_emb_data(emb_dict_file)
print(Fore.BLUE + "EMB DATA LOADED")
print(vocab_dict)

print(Fore.GREEN + "Creating batches")
# prepare data
pre_train_data_loader = Gen_Data_loader(BATCH_SIZE, vocab_dict)
pre_train_data_loader.create_batches([imdb_file_id, sst_pos_file_id, sst_neg_file_id])

gen_data_loader = Gen_Data_loader(BATCH_SIZE, vocab_dict)

gen_data_loader.create_batches([sst_pos_file_id, sst_neg_file_id])

dis_data_loader = Dis_Data_loader(BATCH_SIZE, vocab_dict, MAX_SEQ_LENGTH)

from tensorflow.python.framework import ops
ops.reset_default_graph()

#### build model
#### num_emb, vocab_dict, batch_size, emb_dim, num_units, sequence_length


Load embedding success! Num: 26401
EMB DATA LOADED
{'<PAD>': 0, '<UNK>': 1, '<EOS>': 2, '<GO>': 3, '.': 4, 'the': 5, ',': 6, 'a': 7, 'is': 8, 'and': 9, 'it': 10, 'of': 11, 'to': 12, 'this': 13, 'i': 14, "'s": 15, 'in': 16, 'movie': 17, 'that': 18, '!': 19, 'was': 20, 'but': 21, '?': 22, 'film': 23, 'not': 24, 'for': 25, "n't": 26, 'as': 27, 'he': 28, 'you': 29, 'with': 30, 'are': 31, 'good': 32, 'one': 33, '``': 34, "''": 35, 'all': 36, 'his': 37, 'on': 38, 'be': 39, 'an': 40, 'have': 41, 'what': 42, 'there': 43, 'they': 44, 'just': 45, 'more': 46, 'do': 47, 'great': 48, 'very': 49, 'does': 50, 'so': 51, 'has': 52, 'like': 53, 'at': 54, '...': 55, ')': 56, 'from': 57, 'no': 58, 'really': 59, 'about': 60, '(': 61, 'by': 62, 'well': 63, 'out': 64, 'up': 65, 'some': 66, 'best': 67, 'did': 68, 'its': 69, ':': 70, 'would': 71, 'story': 72, 'also': 73, 'my': 74, 'she': 75, 'me': 76, 'we': 77, 'or': 78, 'can': 79, 'see': 80, 'too': 81, 'will': 82, 'time': 83, 'than': 84, 'much': 85, 'her': 86

In [13]:
generator = Generator(vocab_size, vocab_dict, BATCH_SIZE, EMB_DIM, HIDDEN_DIM, MAX_SEQ_LENGTH)




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.

Instructions for updating:
Use `tf.cast` instead.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
discriminator = Discriminator(sequence_length=MAX_SEQ_LENGTH, num_classes=2,
                              vocab_size=vocab_size,
                              embedding_size=dis_embedding_dim,
                              filter_sizes=dis_filter_sizes, num_filters=dis_num_filters,
                              l2_reg_lambda=dis_l2_reg_lambda)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.




In [15]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())



In [16]:
log = open('save/experiment-log.txt', 'w')

buffer = 'Start pre-training generator...'
print(buffer)
log.write(buffer + '\n')
for epoch in range(120):  #120 #150
    train_loss = pre_train_epoch(sess, generator, pre_train_data_loader)
    if epoch % 5 == 0:
        generate_samples(sess, generator, 1, eval_file, vocab_list, if_log=True, epoch=epoch)
        print('    pre-train epoch ', epoch, 'train_loss ', train_loss)
        buffer = '    epoch:\t' + str(epoch) + '\tnll:\t' + str(train_loss) + '\n'
        log.write(buffer)

Start pre-training generator...
    pre-train epoch  0 train_loss  6.2088313
    pre-train epoch  5 train_loss  3.7855933
    pre-train epoch  10 train_loss  2.8919752
    pre-train epoch  15 train_loss  2.4547174
    pre-train epoch  20 train_loss  2.2449763
    pre-train epoch  25 train_loss  2.1230688
    pre-train epoch  30 train_loss  2.0553548
    pre-train epoch  35 train_loss  2.0040371
    pre-train epoch  40 train_loss  1.9828972
    pre-train epoch  45 train_loss  1.9495916
    pre-train epoch  50 train_loss  1.9484487
    pre-train epoch  55 train_loss  1.9336613
    pre-train epoch  60 train_loss  1.9344183
    pre-train epoch  65 train_loss  1.9150702
    pre-train epoch  70 train_loss  1.9127656
    pre-train epoch  75 train_loss  1.9056684
    pre-train epoch  80 train_loss  1.917424
    pre-train epoch  85 train_loss  1.9213337
    pre-train epoch  90 train_loss  1.9033759
    pre-train epoch  95 train_loss  1.9077492
    pre-train epoch  100 train_loss  1.9091387
    

In [18]:
buffer = 'Start pre-training discriminator...'
print(buffer)
log.write(buffer)

for _ in range(10):   # 10
    generate_samples(sess, generator, 70, negative_file, vocab_list)
    dis_data_loader.load_train_data([sst_pos_file_id, sst_neg_file_id], [negative_file])
    for _ in range(3):
        dis_data_loader.reset_pointer()
        for it in range(dis_data_loader.num_batch):
            x_batch, y_batch = dis_data_loader.next_batch()
            feed = {
                discriminator.input_x: x_batch,
                discriminator.input_y: y_batch,
                discriminator.dropout_keep_prob: dis_dropout_keep_prob,
            }
            d_loss, d_acc, _ = sess.run([discriminator.loss, discriminator.accuracy, discriminator.train_op], feed)
    buffer = "discriminator loss %f acc %f" % (d_loss, d_acc)
    print(buffer)
    log.write(buffer + '\n')

Start pre-training discriminator...
8983
discriminator loss 0.081121 acc 1.000000
8983
discriminator loss 0.051683 acc 1.000000
8983
discriminator loss 0.067046 acc 0.984375
8983
discriminator loss 0.055362 acc 1.000000
8983
discriminator loss 0.026819 acc 1.000000
8983
discriminator loss 0.029798 acc 1.000000
8983
discriminator loss 0.025298 acc 1.000000
8983
discriminator loss 0.054993 acc 0.984375
8983
discriminator loss 0.019822 acc 1.000000
8983
discriminator loss 0.014077 acc 1.000000


In [ ]:
def build_from_ids(vv, vocab_list):
    a = []
    for i in vv:
        a.append(vocab_list[i])
    return(' '.join(a))


In [ ]:
print("Start Adversarial Training...")
log.write('adversarial training...')
for total_batch in range(TOTAL_BATCH):
    # Train the generator
    for it in range(2):
        # print("1")
        samples = generator.generate(sess)
        samples = produce_samples(samples)
        # print("2")
        rewards = generator.get_reward(sess, samples, 16, discriminator)
        # print("3")
        a = str(samples[0])
        b = str(rewards[0])
        # rewards = change_rewards(rewards)
        # c = str(rewards[0])
        d = build_from_ids(samples[0], vocab_list)
        buffer = "%s\n%s\n%s\n\n" % (d, a, b)
        print(buffer)
        log.write(buffer)

        # print("4")
        rewards_loss = generator.update_with_rewards(sess, samples, rewards)
        # print("5")
        # good rewards
        # good_samples = gen_data_loader.next_batch()
        # rewards = np.array([[0.0001] * SEQ_LENGTH] * BATCH_SIZE)
        # a = str(good_samples[0])
        # b = str(rewards[0])
        # buffer = "%s\n%s\n\n" % (a, b)
        # print(buffer)
        # log.write(buffer)
        # rewards_loss = generator.update_with_rewards(sess, good_samples, rewards, START_TOKEN)

        # little1 good reward
        little1_samples = gen_data_loader.next_batch()
        rewards = generator.get_reward(sess, little1_samples, 16, discriminator)
        a = str(little1_samples[0])
        b = str(rewards[0])
        buffer = "%s\n%s\n\n" % (a, b)
        # print(buffer)
        log.write(buffer)
        rewards_loss = generator.update_with_rewards(sess, little1_samples, rewards)

    # generate_infer(sess, generator, epoch, vocab_list)

    # Test
    if total_batch % 5 == 0 or total_batch == TOTAL_BATCH - 1:
        generate_samples(sess, generator, 120, eval_file, vocab_list, if_log=True)
        generate_infer(sess, generator, total_batch, vocab_list)
        # generate_samples(sess, generator, BATCH_SIZE, generated_num, eval_file)
        # likelihood_data_loader.create_batches(eval_file)
        # test_loss = target_loss(sess, target_lstm, likelihood_data_loader)
        buffer = 'reward-train epoch %s train loss %s' % (str(total_batch), str(rewards_loss))
        print(buffer)
        log.write(buffer + '\n')
        generator.save_model(sess)

    # Train the discriminator
    begin = True
    for _ in range(1):
        generate_samples(sess, generator, 70, negative_file, vocab_list)
        dis_data_loader.load_train_data([sst_pos_file_id, sst_neg_file_id], [negative_file])
        for _ in range(3):
            dis_data_loader.reset_pointer()
            for it in range(dis_data_loader.num_batch):
                x_batch, y_batch = dis_data_loader.next_batch()
                feed = {
                    discriminator.input_x: x_batch,
                    discriminator.input_y: y_batch,
                    discriminator.dropout_keep_prob: dis_dropout_keep_prob,
                }
                d_loss, d_acc, _ = sess.run([discriminator.loss, discriminator.accuracy, discriminator.train_op],
                                            feed)
                if (total_batch % 5 == 0 or total_batch == TOTAL_BATCH - 1) and begin:
                    buffer = "discriminator loss %f acc %f\n" % (d_loss, d_acc)
                    print(buffer)
                    log.write(buffer)
                    begin = False

    # pretrain
    for _ in range(10):
        train_loss = pre_train_epoch(sess, generator, pre_train_data_loader)


Start Adversarial Training...
crock and bullsh*t ego rub story .
[16889, 9, 20280, 4064, 9838, 72, 4]
[0.65307844 0.78227353 0.64465797 0.58697027 0.5897248  0.63964564
 0.60400605 0.48647422 0.5957587  0.63486135 0.63486135 0.63486135
 0.63486135 0.63486135 0.63486135 0.63486135 0.63486135]


when lighting strikes
[104, 2408, 2298]
[0.9045175 0.9779308 0.9911349 0.9936077 0.9256228 0.9998322 0.9998322
 0.9998322 0.9998322 0.9998322 0.9998322 0.9998322 0.9998322 0.9998322
 0.9998322 0.9998322 0.9998322]


save/infer/0.txt saves
reward-train epoch 0 train loss 11.184036
save model success!
discriminator loss 0.253857 acc 0.921875

a solid action film
[7, 366, 94, 23]
[0.8687782  0.982757   0.9952085  0.9019437  0.9014134  0.84762114
 0.9999237  0.9999237  0.9999237  0.9999237  0.9999237  0.9999237
 0.9999237  0.9999237  0.9999237  0.9999237  0.9999237 ]


nice action
[293, 94]
[0.9183455  0.9458398  0.832018   0.95840806 0.9998138  0.9998138
 0.9998138  0.9998138  0.9998138  0.9998138  